<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/Memory/LangMem04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langmem

In [2]:
from google.colab import userdata
import os

# 사용자에게 안전하게 API 키 입력 받기
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')

# 환경 변수로 설정
os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

In [3]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langmem import create_manage_memory_tool, create_search_memory_tool

# Create tool with {user_id} template
tool = create_manage_memory_tool(namespace=("memories", "{user_id}"))
# Agent just sees that it has memory. It doesn't know where it's stored.
app = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[tool])
# Use with different users
app.invoke(
    {"messages": [{"role": "user", "content": "I like dolphins"}]},
    config={"configurable": {"user_id": "user-123"}}
)  # Stores in ("memories", "user-123")

In [ ]:
# Organization-level
tool = create_manage_memory_tool(
    namespace=("memories", "{org_id}")
)
app = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[tool])
app.invoke(
    {"messages": [{"role": "user", "content": "I'm questioning the new company health plan.."}]},
    config={"configurable": {"org_id": "acme"}}
)

# User within organization
tool = create_manage_memory_tool(
    namespace=("memories", "{org_id}", "{user_id}")
)
# If you wanted to, you could let the agent
# search over all users within an organization
tool = create_search_memory_tool(
    namespace=("memories", "{org_id}")
)
app = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[tool])
app.invoke(
    {"messages": [{"role": "user", "content": "What's our policy on dogs at work?"}]},
    config={"configurable": {"org_id": "acme", "user_id": "alice"}}
)

# You could also organize memories by type or category if you prefer
tool = create_manage_memory_tool(
    namespace=("agent_smith", "memories", "{user_id}", "preferences")
)
app = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[tool])
app.invoke(
    {"messages": [{"role": "user", "content": "I like dolphins"}]},
    config={"configurable": {"user_id": "alice"}}
)